Vector Space Model คือการคำนวณหาค่าความเหมือน หรือ เปรียบเทียบข้อมูล โดยใช้วิธีการคำนวณแบบ cosine ซึ่ง ผลลัพท์ที่ได้ จะมีค่าจาก 0 ถึง 1

ยิ่งผลลพท์ที่ได้ มีค่าเข้าใกล้ 0 เท่าไร นั่นหมายถึง ข้อมูลที่เปรียบเทียบกัน เหมือนกันมากเท่านั้น (0 หมายถึงข้อมูลเดียวกันเลย)

ปัญหาของ Vector space model ไม่สามารถให้ความสำคัญในการค้นหากลุ่มคำ หรือ ลำดับของคำได้

In [1]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")

#sentence = "SWU is a university located in downtown"
#words = nltk.tokenize.word_tokenize(sentence)
#tokens = [word.lower() for word in words if word not in nltk.corpus.stopwords.words("english")]
#print(tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/teerawitseekasamit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/teerawitseekasamit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from nltk.stem import PorterStemmer

#porter = PorterStemmer ()
#print(porter.stem("secretary"))

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
#https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics
df = pd.read_csv('lyrics.csv')

In [4]:
del df['index'],df['year'],df['artist']

In [5]:
df = df.sample(n = 100)

In [6]:
df = df[pd.notnull(df['lyrics'])]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 297669 to 157543
Data columns (total 3 columns):
song      72 non-null object
genre     72 non-null object
lyrics    72 non-null object
dtypes: object(3)
memory usage: 2.2+ KB


In [8]:
df.dtypes

song      object
genre     object
lyrics    object
dtype: object

In [9]:
df.head(10)

,song,genre,lyrics
297669,my-arms-keep-missing-u,Electronic,You know I've been so lonely every night\nSinc...
1999,constantinople,Rock,Across the western world\nThe fights are going...
13913,the-work-of-art,Rock,A heavy feeling creeps through my bones\nAs I ...
141442,disarm,Country,Disarm you with a smile\nAnd cut you like you ...
183852,i-believe-in-santa-claus,Not Available,(Dolly Parton)\nI believe in Santa Claus\nI be...
32423,walk-away-remix,Not Available,Chorus\nIf I told you that I loved you would y...
352268,this-is-cracker-soul,Rock,"Hey hey, it's okay to make\nA little mess out ..."
262942,another-me,Rock,Remember when I sang that song to you\nAnd you...
301734,it-s-only-a-paper-moon-i-like-him,Pop,"Fanny\nSay it's only a paper moon, sailing ove..."
144942,waking-dream,R&B,"Suddenly, don't know from where you came\nLike..."


In [10]:
lyrics = list(df["lyrics"])

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(decode_error='replace', encoding='utf-8')

In [12]:
v

TfidfVectorizer(analyzer='word', binary=False, decode_error='replace',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [13]:
df['genre'].value_counts(dropna=True)

Rock             34
Pop              10
Hip-Hop           8
Not Available     6
Country           5
Metal             4
Electronic        2
Indie             1
Jazz              1
R&B               1
Name: genre, dtype: int64

In [14]:
ly_store = list(df["lyrics"].fillna(method='ffill'))

In [15]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(ly_store)
tfidf = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names()).transpose()
tfidf.columns = list(df["song"])

In [16]:
urfl = input("How do you feel ?")

How do you feel ?sad


In [17]:
query = urfl
query_vector = vectorizer.transform([query])
q_query = pd.DataFrame(query_vector.toarray(), columns = vectorizer.get_feature_names()).transpose()

In [18]:
print(q_query)

                 0
12             0.0
20             0.0
2015           0.0
21st           0.0
23             0.0
2x             0.0
30             0.0
37             0.0
3rd            0.0
528            0.0
54th           0.0
5h             0.0
5th            0.0
70             0.0
80             0.0
86             0.0
90             0.0
91             0.0
aaaction       0.0
ab             0.0
abandon        0.0
abba           0.0
abend          0.0
aber           0.0
abgebrochen    0.0
abgelutschter  0.0
abgenommen     0.0
abgeschleckt   0.0
abgesetzt      0.0
abide          0.0
...            ...
yesterday      0.0
yet            0.0
yi             0.0
yippie         0.0
yo             0.0
you            0.0
young          0.0
your           0.0
youre          0.0
yours          0.0
yourself       0.0
zag            0.0
zappa          0.0
zaster         0.0
zeeky          0.0
zehn           0.0
zeit           0.0
ziel           0.0
zig            0.0
zone           0.0
zoo         

In [19]:
from scipy.spatial.distance import cdist

sim = 1 - cdist(tfidf.values.T, query_vector.toarray(), metric = "cosine")

for i in range(len(sim)):
    if sim[i] == np.max(sim):
        recsong = tfidf.columns[i]
print("Recommend for your song is " , recsong)

Recommend for your song is  chewing-gum
